In [708]:
import pandas as pd
import numpy as np
import csv
from geneticalgorithm import geneticalgorithm as ga
import random
import math
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import copy
import time

In [690]:
############# Creates Fitness Values for Path edge "Take input a selection matrix that contains all city path combinations"
def calculatingFitness(): 
    #Creates selection_fitness which contains fitness information for each tour 
    selection_fitness=[]
    for j in range(0,pop_size):
        total=0
        for i in range(0,len(df)-1):
            total=total+fitness[selection[j][i]][selection[j][i+1]]
        selection_fitness.append(total)
    df_2 = pd.DataFrame(selection_fitness,columns=['pathTotalFitness'])
    df_2["pathNo"]=df_2.index  
    #Contains Fitness value vs Tour No(every tour number to be mapper with some tour)
    return df_2

In [691]:
def rouletteSelection(df_2):
    ######### Selection Module
    df_2.sort_values(by=['pathTotalFitness'],ascending=False,inplace=True)
    df_2.reset_index(inplace=True)
    del df_2["index"] 
    df_2["Prob"]=df_2["pathTotalFitness"]/(df_2["pathTotalFitness"].sum())
    df_2["Prob"].sum()
    df_2["Expected_count"]=df_2["Prob"]*pop_size
    new=[]
    i=1
    while(i<=pop_size):
        new.append(df_2["pathNo"][0])
        #Subtracting 0.5 from Expected count for selection, to giv other strings a chance to be selected
        df_2["Expected_count"][0]=df_2["Expected_count"][0]-0.5
        df_2.sort_values(by=['Expected_count'],ascending=False,inplace=True)
        df_2.reset_index(inplace=True)
        del df_2["index"]
        i=i+1
    try:
        if new_df is not None:      #Checking if DataFrame already exists
            del new_df    
    except:
        pass
    # new_df contains selected set of paths
    new_df = pd.DataFrame(new,columns=['pathNo']) 
    return new_df

In [692]:
def crossover(new_df,choice):
    ####### All Crossover

    #Selecting Paths on which to apply Crossover by Probability
    new_df["random_prob"]=np.random.uniform(low=0, high=1, size=pop_size)
    new_df["Crossover_eligibility"]=new_df["random_prob"]<=pc      # Checking each population valuw with Pc (Crossover Probability)
    try:
        if Sel_crossover is not None:      #Checking if DataFrame already exists
            del Sel_crossover    
    except:
        pass

    # Divided population pool into 2 parts, one part that is eligible for Crossover (Randomly) and other that is not
    Sel_crossover=new_df[new_df["Crossover_eligibility"]==True]["pathNo"]
    seln=Sel_crossover.to_list()
    try:
        if NotSel_crossover is not None:      #Checking if DataFrame already exists
            del NotSel_crossover    
    except:
        pass
    NotSel_crossover=new_df[new_df["Crossover_eligibility"]==False]["pathNo"]
    notseln=NotSel_crossover.to_list()
    random.shuffle(seln)              #Shufling the list for proper crossover 

    #Checking if eligible strings are odd, so we will have to remove one string from crossover queue
    if (len(seln)%2!=0):
        notseln.append(seln.pop())

    ############################################ PMX Crossover

    #print "Total Paths",len(new_df),"# Paths for Crossover",len(seln)
    while len(seln)>0:
        point1=random.randint(1,len(df)-1)
        point2=random.randint(point1,len(df)-1)
        parent1=seln.pop()
        parent2=seln.pop()        
        #print "Iteration",len(seln),"Cross-point1",point1, "Cross-point2",point2 , "Path_No_1",parent1, "Path_No_2nd", parent2
        p1=selection[parent1]
        p2=selection[parent2]
    ############################ Choose Any one Crossover here   
        if choice==1:
            PMX_crossover(point1,point2,p1,p2)
        elif choice==2:    
            ox1_crossover(point1,point2,p1,p2)
        elif choice==3:
            ox2_crossover(point1,point2,p1,p2)
        else:
            pos_crossover(point1,point2,p1,p2)
        
    ### Converting Nonselections into city wise tour paths
    try:
        for i in notseln:
            child_crossover.append(copy.deepcopy(selection[i]))
    except:
        pass
        



In [693]:
def ox1_crossover(point1,point2,p1,p2):
    
    #"Parent 1", p1,"Parent 2",p2
    c1=copy.deepcopy(p1)
    c2=copy.deepcopy(p2)
    list1=[]
    list2=[]

    #One Offspring
    k=point2
    for i in range(0,len(p1)):
        list1.append(p1[k])
        k=k+1
        if k==len(p1):
            k=0
        if k==point2:
            break

    list1_final = [i for j, i in enumerate(list1) if i not in c2[point1:point2]] 
    k=point2
    for i in range(0,len(p1)-(point2-point1)):
        c2[k]=list1_final[i]
        k=k+1
        if k==len(c2):
            k=0
        if k==point1:
            break

    # 2nd Offspring
    k=point2
    for i in range(0,len(p2)):
        list2.append(p2[k])
        k=k+1
        if k==len(p2):
            k=0
        if k==point2:
            break
    list2_final = [i for j, i in enumerate(list2) if i not in c1[point1:point2]] 
    k=point2
    for i in range(0,len(p1)-(point2-point1)):
        c1[k]=list2_final[i]
        k=k+1
        if k==len(c1):
            k=0
        if k==point1:
            break
    #print "Child1",c1, "Child2",c2

    child_crossover.append(copy.deepcopy(c1))
    child_crossover.append(copy.deepcopy(c2))

In [694]:
def ox2_crossover(point1,point2,p1,p2):
    #"Parent 1", p1,"Parent 2",p2
    c1=copy.deepcopy(p1)
    c2=copy.deepcopy(p2)
    list1=[]
    list2=[]
    fix=np.random.randint(low = 1,high=len(p1)-1)
    x = random.sample(range(0, len(p2)-1), fix)

    #One Child
    for i in x:
        list1.append(p2[i])
    for i in range(0,len(list1)):
        c1[c1.index(list1[i])]=9999
    list1_final = [i for j, i in enumerate(p2) if i not in c1[:]] 
    for i in range(0,len(c1)):
        if c1[i]==9999:
            c1[i]=list1_final.pop()

    #Second Child
    for i in x:
        list2.append(p1[i])
    for i in range(0,len(list2)):
        c2[c2.index(list2[i])]=9999

    list2_final = [i for j, i in enumerate(p1) if i not in c2[:]] 
    for i in range(0,len(c2)):
        if c2[i]==9999:
            c2[i]=list2_final.pop() 

    #print "Child1",c1, "Child2",c2        
    child_crossover.append(copy.deepcopy(c1))
    child_crossover.append(copy.deepcopy(c2))

In [695]:
def pos_crossover(point1,point2,p1,p2):
    #"Parent 1", p1,"Parent 2",p2
    c1=copy.deepcopy(p1)
    c2=copy.deepcopy(p2)
    list1=[]
    list2=[]
    
    #Swapping the values for random indices created in x
    fix=np.random.randint(low = 1,high=len(p1))
    x = random.sample(range(0, len(p1)-1), fix)
    for i in x:
        temp=c1[i]
        c1[i]=c2[i]
        c2[i]=temp
        list1.append(c1[i])
        list2.append(c2[i])
    #print c1,c2
    
    # Marking the places at each offspring so as to work on them
    k=range(0,len(p1))
    left = [i for j, i in enumerate(k) if i not in x[:]]  
    for i in left:
        c1[i]=9999
        c2[i]=9999
    #print c1,c2
    # Creates an ordered list of cities that are to be placed at positions marked earlier
    list1_final = [i for j, i in enumerate(p1) if i not in list1[:]]
    list2_final = [i for j, i in enumerate(p2) if i not in list2[:]]
    
    q=0
    w=0
    for i in range(0,len(c1)):
        if c1[i]==9999:
            #print i
            c1[i]=list1_final[q]
            q=q+1
        if c2[i]==9999:
            c2[i]=list2_final[w]
            w=w+1
    #print "Child1",c1, "Child2",c2        
    child_crossover.append(copy.deepcopy(c1))
    child_crossover.append(copy.deepcopy(c2))


In [791]:
def PMX_crossover(point1,point2,p1,p2):
    #print "Parent 1", p1,"Parent 2",p2
    c1=copy.deepcopy(p2)
    c2=copy.deepcopy(p1)
    relations=[]
    
    #Creating Relations
    for i in range(point1,point2+1):
        relations.append([p1[i], p2[i]])
    #print "Relations------",relations
    
    # First Child
    for i in range(0,10):
        new_relations=relations
        
        #Checking if the element is not related to some other element
        if (p1[i] not in c1[point1:point2+1]) and (i not in range(point1,point2+1)):
            c1[i]=p1[i]
        # Proceeding here if element present is related
        elif (p1[i] in c1[point1:point2+1]) and (i not in range(point1,point2+1)):  
            flag=1
            err=0
            while flag==1:
                err=err+1
                if err>200:
                    break
                for x in range(0,len(c1[point1:point2+1])):
                    if p1[i]==new_relations[x][1]:
                        c1[i]=new_relations[x][0]
                        #print c1
                        break
                # To make sure that duplicates are not present
                if c1[i] not in c1[0:i]:
                    flag=0                        
                if c1[i] in c1[point1:point2+1]:
                    p1[i]=c1[i]
                    flag=1                   
                else:
                    flag=0
                    break     
    #print "Final c1",c1
    child_crossover.append(copy.deepcopy(c1))
    
    # Second Child
    for i in range(0,10):
        new_relations=relations
        if (p2[i] not in c2[point1:point2+1]) and (i not in range(point1,point2+1)):
            c2[i]=p2[i]
        elif (p2[i] in c2[point1:point2+1]) and (i not in range(point1,point2+1)):  
            flag=1
            err=0
            while flag==1:   
                err=err+1
                if err>200:
                    break
                for x in range(0,len(c2[point1:point2+1])):
                    if p2[i]==new_relations[x][0]:
                        c2[i]=new_relations[x][1]
                        #print c2
                        break
                if c2[i] not in c2[0:i]:
                    flag=0
                if c2[i] in c2[point1:point2+1]:
                    p2[i]=c2[i]
                    flag=1                    

                else:
                    flag=0
                    break   
    #print "Final c2",c2     
    child_crossover.append(copy.deepcopy(c2))

                        

In [697]:
def mutation():
    #Performing Exhange Mutation
    df_mut=pd.DataFrame() 
    df_mut["random_prob"]=np.random.uniform(low=0, high=1, size=pop_size)
    df_mut["pathNo"]=df_mut.index
    df_mut["Crossover_eligibility"]=df_mut["random_prob"]<=pm
    #Selecting the ones that need to be mutated
    df_mut_true=df_mut[df_mut["Crossover_eligibility"]==True]
    mut_list=df_mut_true["pathNo"].to_list()

    NotSel_mutation=df_mut[df_mut["Crossover_eligibility"]==False]["pathNo"]
    notseln_mut=NotSel_mutation.to_list()
    #print "NotMut",notseln_mut

    try:    
        for i in notseln_mut:
            child_mutation.append(copy.deepcopy(selection[i]))
    except:
        pass
    i=0
    temp=0
    while i <len(mut_list):
        l1=[]
        pop1=mut_list.pop()
        l1=copy.deepcopy(selection[pop1])
        point1=random.randint(1,len(df)-1)
        point2=random.randint(point1,len(df)-1)
        temp=l1[point1]
        l1[point1]=l1[point2]
        l1[point2]=temp
        i=i+1
        child_mutation.append(copy.deepcopy(l1))
        #print "Mutated-",l1



In [701]:
#### Provide Iterations
iterations=10
#For selecting the Crossover Choice
#4- POS Crossover
#3- OX2 Crossover
#2- OX1 Crossover
#1- PMX Crossover

for choice in [4,3,2,1]:
    print "Crossover Choice-------", choice
    ts = time.time()
    pc=0.95
    pm=0.01
    bestFitness=[]
    bestPath=[]

    #Taking City coordinates as input
    df = pd.read_csv('City_Coordinates.txt', sep=",", header=None)
    df.columns=["lat","long"]
    df["cityNo"]=df.index
    df.reset_index(level=0, inplace=True)
    del df["index"]
    
    #Calulating Fitness by (1/eucledian distance)
    fitness=[[0 for i in range(len(df))]]*len(df) 
    for i in range(0,len(df)):
        for j in range(i+1,len(df)):
            distance=(((df.get_value(i,"lat")-df.get_value(j,"lat"))**2)+((df.get_value(i,"long")-df.get_value(j,"long"))**2))**0.5
            fitness[i][j]=1/distance
            fitness[j][i]=fitness[i][j]
    pop_size=50


    ######### Intital Population Initialization
    path=[i for i in range(len(df))]
    selection=[]

    for i in range(0,len(df)):
        random_p=np.random.permutation(path).tolist()
        selection.append(copy.deepcopy(random_p))
    selection_copy=copy.deepcopy(selection)

    Generations=0
    for itera in range(0,iterations):
        #Caluculating Fitness
        df2=calculatingFitness()
        #Roulette Selection
        new_df=rouletteSelection(df2)
        #Crossover
        child_crossover=[]
        crossover(new_df,choice)
        #Mutation
        selection=[]
        selection=copy.deepcopy(child_crossover)
        child_mutation=[]
        mutation()
        selection=copy.deepcopy(child_mutation)
        del df2
        del new_df
        df2=calculatingFitness()
        #Finding best Fitness and Tour
        bestFitness.append(max(df2['pathTotalFitness']))
        k=df2[df2['pathTotalFitness']==df2['pathTotalFitness'].max()]
        k=k.reset_index()
        bestPath.append(copy.deepcopy(selection[k.get_value(0,"pathNo")]))
        Generations=Generations+1
        print "Iteration Number-",Generations
        del df2
    print "Best Fitness Value",bestFitness[iterations-1]
    print "Best Path for Cities to cover",bestPath[iterations-1]
    #To calculate Run Time
    ts1 = time.time()
    print "Seconds",ts1-ts

Crossover Choice------- 1


/home/prakhar/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:19: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/prakhar/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/prakhar/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:53: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Iteration Number- 1
Iteration Number- 2


KeyboardInterrupt: 